In [1]:
import os
try:
    os.remove('snorkel.db')
except:
    pass

In [2]:
from snorkel import SnorkelSession
session = SnorkelSession()

import os, sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

In [3]:
import os
from snorkel.parser import CorpusParser
from snorkel.parser import HTMLParser
from snorkel.parser import OmniParser

file_name = os.environ['SNORKELHOME'] + '/tutorials/tables/sandbox/diseases.xhtml'
doc_parser = HTMLParser(path=file_name)
context_parser = OmniParser(tabular=True, 
                            lingual=True, blacklist=['caption'], 
                            flatten=['span','br','i'], flatten_delim='')
#                             replacements=[('Temperature', 'Temp')])
cp = CorpusParser(doc_parser, context_parser, max_docs=1)

In [4]:
# from bs4 import BeautifulSoup, NavigableString, Comment
# with open(file_name, 'rb') as f:
#     html_doc = f.read()
# soup = BeautifulSoup(html_doc, 'html.parser')

In [5]:
# for child in soup.body.children:
#     print isinstance(child, Comment)
# print isinstance(soup.table.tr, Comment)

In [6]:
%time corpus = cp.parse_corpus(name='Sandbox', session=session)
print "%d Phrases in %s" % (
    len([phrase for doc in corpus.documents for phrase in doc.phrases]), corpus)

[========================================] 100%
CPU times: user 160 ms, sys: 20.2 ms, total: 180 ms
Wall time: 466 ms
35 Phrases in Corpus (Sandbox)


In [7]:
for p in corpus.documents[0].phrases:
    print p.text

Types of viruses, coughs, and colds
Here isa line break
I don't have Brain Canceror the hiccups
See Table 1 Below.
See Table 2 Below.
SeeTable 3Below.
Common Ailments
Disease
Location
Year
Polio and BC546 is -55OC cold.
-Dublin to Milwaukee
?
2001
I don't like Chicken Pox or pizza.
Shingles is also bad.
whooping cough
Scurvy
Annapolis
Junction and Storage Temperature -55 to 150 o ?
C
In between the tables there is a nasty case of heart attack
Problem
Cause
Cost
Arthritis
Pokemon Go
Free
YellowFever
Unicorns
$17.75
Hypochondria
Fear
$100
And here is a final sentence with warts.


In [ ]:
from snorkel.lf_helpers import _get_node
from lxml.html import fromstring
from lxml import etree

p = corpus.documents[0].phrases[0]
print p.xpath
print p.document.text[:10]
tree = etree.ElementTree(fromstring(p.document.text))
tree.xpath('/html/body/table[1]')
# print type(p.document.etree.xpath('/html'))
# .xpath(p.xpath[1:])
print _get_node(p)

In [ ]:
for p in corpus.documents[0].phrases:
#     print (p.table, p.cell)
        print (p.html_tag, p.xpath)

In [ ]:
xpath = '/html/body/table[1]/tbody/tr[5]/td[2]'


In [ ]:
# for i in range(34):
#     print corpus.documents[0].phrases[i].is_tabular()
p = corpus.documents[0].phrases[5]
print p.is_lingual()
print p.is_tabular()
print p.is_visual()
print p.is_structural()
# # print [phrase for phrase in corpus.documents[0].phrases]

In [ ]:
# for cell in corpus.documents[0].cells: print cell.text[:10]

In [ ]:
# (a,b,c) = (2,(3 if True else 5) ,4)

In [ ]:
from snorkel.models import candidate_subclass

# Year = candidate_subclass('Year', ['year'])
# Temp = candidate_subclass('Temp', ['temp'])
# Disease = candidate_subclass('Disease', ['disease'])
# Part = candidate_subclass('Part', ['part'])
Disease_Part = candidate_subclass('Disease_Part', ['disease','part'])

In [ ]:
from snorkel.candidates import OmniNgrams
from hardware_utils import OmniNgramsPart, OmniNgramsTemp

omni_ngrams = OmniNgrams(n_max=3, split_tokens=None)
omni_part = OmniNgrams(n_max=1)
# omni_part = OmniNgramsPart(n_max=1)
# omni_temp = OmniNgramsTemp(n_max=3)
# omni_temp = OmniNgrams(n_max=2)

In [ ]:
diseases = ['viruses', 'coughs', 'colds', 'brain', 'cancer', 'brain cancer', 'hiccups', 
            'ailments', 'disease', 'polio', 'chicken pox', 'shingles', 'whooping cough',
            'scurvy', 'infectious diseases', 'heart attack', 'arthritis', 'fever', 
            'yellow fever', 'hypochondria', 'pneumonia', 'warts']
print "Loaded %d diseases." % len(diseases)

In [ ]:
from snorkel.matchers import DictionaryMatch, RegexMatchEach, RegexMatchSpan, DateMatcher

disease_matcher = DictionaryMatch(d=diseases, ignore_case=True)
# year_matcher = DateMatcher()
part_matcher = RegexMatchEach(rgx='BC.*')
# temp_matcher = RegexMatchSpan(rgx=r'-[5-7][05]')

In [ ]:
from snorkel.candidates import CandidateExtractor

# ce = CandidateExtractor(Part, [omni_part], [part_matcher])
# ce = CandidateExtractor(Disease, [omni_ngrams], [disease_matcher])
# ce = CandidateExtractor(Temp, [omni_temp], [temp_matcher])
ce = CandidateExtractor(Disease_Part, 
                        [omni_ngrams, omni_part], 
                        [disease_matcher, part_matcher])

In [ ]:
%time candidates = ce.extract(corpus.documents, 'Sandbox Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

In [ ]:
for c in candidates:
    print c

In [ ]:
from snorkel.features.features import get_all_feats
c = candidates[4]
print c
print
for f, _ in get_all_feats(c):
    print f

In [ ]:
from snorkel.lf_helpers import *

s = c[0]
print s
print get_tag(s)
print get_parent_tag(s)
print get_prev_sibling_tags(s)
print get_prev_sibling_tags(s)
print get_next_sibling_tags(s)
print get_ancestor_class_names(s)
print get_ancestor_tag_names(s)
print get_ancestor_id_names(s)

In [ ]:
from snorkel.lf_helpers import *

# print [cell.text[:8] for cell in _get_aligned_cells(root_phrase.cell, axis, infer=infer)]
# print [phrase.text for cell in _get_aligned_cells(root_phrase.cell, axis, infer=infer) for phrase in cell.phrases]
print c.disease.parent
print c.part.parent
print list(get_left_ngrams(c.part, n_max=1, window=10, lower=False))
print list(get_right_ngrams(c.part, n_max=1, window=5))
print list(get_phrase_ngrams(c.part, n_max=1))
print list(get_cell_ngrams(c.part, n_max=1))
print list(get_row_ngrams(c.part, n_max=1, direct=True, infer=True))
print list(get_head_ngrams(c.part, 'row'))
print list(get_head_ngrams(c.part, 'col'))

In [ ]:
from snorkel.models import CandidateSet
train = CandidateSet(name='train', candidates=candidates[:25])
dev = CandidateSet(name='test', candidates=candidates[25:])

In [ ]:
from snorkel.fast_annotations import FeatureManager

feature_manager = FeatureManager()
%time F_train = feature_manager.create(session, train, 'Sandbox Features')
F_train

# from snorkel.features import get_span_feats
# c = candidates[10]
# %prun for feat in get_span_feats(c): print feat

In [ ]:
from snorkel.utils import get_keys_by_candidate

c = candidates[1]
print c
print
for f in get_keys_by_candidate(c, F_train)[:]: print f

In [ ]:
# c = candidates[0]
# print c
# print c.year
# print c.year.parent
# print c.year.char_start
# print c.year.char_end
# print c.year.parent.words
# print c.year.parent.char_offsets
# print c.year.parent.lemmas
# print c.year.parent.pos_tags
# print c.year.parent.ner_tags
# print c.year.parent.dep_parents
# print c.year.parent.dep_labels
# print c.year.get_word_start()
# print c.year.get_word_end()
# # print get_row_ngrams(c.year)
# # print get_row_ngrams(c.year, infer=True)
# %time for i in range(1000): _get_aligned_cells((c.year).parent.cell, 'col', infer=True)

In [ ]:
# from snorkel.models import Span, ImplicitSpan, TemporarySpan

# print isinstance(c.year, TemporarySpan)

In [ ]:
# from snorkel.lf_helpers import get_between_ngrams, get_left_ngrams, get_right_ngrams
# from snorkel.lf_helpers import contains_token, contains_regex
# from snorkel.lf_helpers import get_phrase_ngrams, get_cell_ngrams, get_neighbor_cell_ngrams
# from snorkel.lf_helpers import get_row_ngrams, get_col_ngrams, get_aligned_ngrams
# from snorkel.lf_helpers import same_document, same_table, same_cell, same_phrase
# from snorkel.lf_helpers import _get_aligned_cells, _get_nonempty_cell

In [ ]:
from snorkel.lf_helpers import *
c = candidates[2]
print list(get_head_ngrams(c.disease, axis='col'))
# print get_left_ngrams(c.disease)
# print get_right_ngrams(c.disease)
# print contains_token(c, 'plague')
# print contains_regex(c, r'pla')
# print same_document(c)
# print same_table(c)
# print same_cell(c)
# print same_phrase(c)
# print get_phrase_ngrams(c.disease, n_min=1, n_max=3, case_sensitive=True)
# print get_cell_ngrams(c.disease, attrib='pos_tags')
# print get_neighbor_cell_ngrams(c.disease, dist=2, directions=True)
# print get_row_ngrams(c.disease)
# print get_col_ngrams(c.disease)
# print get_aligned_ngrams(c.disease)
# print get_aligned_ngrams(c.disease, infer=True)

In [ ]:
# from hardware_utils import expand_implicit_text 

# for part in expand_implicit_text(''.join(['BC856/857/858', '/', '859/860'])): print part

In [ ]:
# def foo(n):
#     phrase = 'repeat me'
#     pmul = phrase * n
#     pjoi = ''.join([phrase for x in xrange(n)])
#     pinc = ''
#     for x in xrange(n):
#         pinc += phrase
#     del pmul, pjoi, pinc

In [ ]:
# %lprun -f foo foo(1000)

In [ ]:
# from hardware_utils import get_gold_dict

# filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# gold_dict = get_gold_dict(filename, 'stg_temp_min')

In [ ]:
# print len(gold_dict.values())
# print gold_dict.values().count(1)

In [ ]:
# from hardware_utils import count_hardware_labels

# filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# %time (certain, maybe) = count_hardware_labels(loader, candidates, filename, attrib='stg_temp_min', attrib_class='temp')

In [ ]:
# %time 
# for i in range(10000): 
#     for j in range(100):
#         1 + 1

In [ ]:
# from snorkel.utils import ProgressBar;

In [ ]:
# %%time
# n = 100000
# pb = ProgressBar(n)
# for i in xrange(n):
#     pb.bar(i)
# pb.close()

In [ ]:
# N = 235
# N = 10
# ticks = set([int(i * N/100.0) for i in range(1,101)])
# print ticks

In [ ]:
for doc, text in doc_parser.parse():
    pass

In [ ]:
print text

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(text, 'lxml')

In [ ]:
def create_xpath(tags, counts):
    xpath = ''
    for i, tag in enumerate(tags):
        xpath += '/' + tag
        if counts[i] != 1:
            xpath += '[%d]' % (counts[i] - 1)
    return xpath

In [ ]:
tags = ['html','body','table','tr','th']
counts = [1, 1, 2, 2, 1]
print create_xpath(tags, counts)

In [ ]:
print soup.html.body.table.next_element

In [ ]:
from lxml.html import fromstring
import lxml

html = lxml.html.fromstring(text)
# htmlparser = etree.HTMLParser()
# xml.etree.ElementTree.fromstring(text)
# tree = lxml.etree.fromstring(text, htmlparser)
# tree.xpath(xpathselector)

In [ ]:
etree.tostring(tree)

In [ ]:
print tree.xpath('/html/body/table[1]')
print tree.xpath('/html/body/table[1]')

In [ ]:
for noe in 